<a href="https://colab.research.google.com/github/YB-nt/job_scraping/blob/main/nlp/job_scrap_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy > /dev/null
!pip install nltk > /dev/null
!pip install twython > /dev/null
!pip install psycopg2 >/dev/null

In [ ]:
from konlpy.tag import Okt
import psycopg2
from collections import Counter
import re

import nltk
nltk.download('all')

from konlpy.tag import Twitter; t = Twitter()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [ ]:
_host ="arjuna.db.elephantsql.com"
_database="xbegavim"
_user ="xbegavim"
_password ="m7_4leTxqwHlcCKKYhhuL3SXO2dHUmo5"
conn = psycopg2.connect(
        host=_host,
        database=_database,
        user=_user,
        password=_password)
cur = conn.cursor()

temp_main =[]
temp_require =[]
temp_common=[]
temp_pt=[]


In [ ]:
# count =0
for col_name in ["job_main","require","common","pt"]:
    cur.execute(f"SELECT {col_name} FROM split_data")
    col_data = cur.fetchall()
    temp_count =[]
    temp_list = []
    for idx,data in enumerate(col_data):
        
        if("정보" in data[0] or "없음" in data[0]):
            continue
        if(idx==0):
            temp_list = temp_main
        elif(idx==1):
            temp_list = temp_require
        elif(idx==2):
            temp_list = temp_common
        elif(idx==3):
            temp_list = temp_pt
        # 영어 counter
        _data = data[0]
        counter = Counter(t.morphs(_data))
        eng_data = [i for i in counter.most_common() if i[0].encode().isalpha()]
        for i in eng_data:
            if(len(i[0])>10):
                eng_data.remove(i)
                counter.update(re.findall('[A-Z][^A-Z]*',i[0]))
        ko_data = Counter(Okt().nouns(_data)).most_common(len(eng_data))
        temp_list.extend(eng_data)        
        temp_list.extend(ko_data)        


In [ ]:
sorted_main = sorted(list(set(d for d in temp_main if len(d[0])>1)),key=lambda x :-x[-1])
sorted_require = sorted(list(set(d for d in temp_require if len(d[0])>1)),key=lambda x :-x[-1])
sorted_common = sorted(list(set(d for d in temp_common if len(d[0])>1)),key=lambda x :-x[-1])
sorted_pt = sorted(list(set(d for d in temp_pt if len(d[0])>1)),key=lambda x :-x[-1])



In [ ]:
print(sorted_main)
print(sorted_common)
print(sorted_require)
print(sorted_pt)

[('경험', 4), ('서비스', 3), ('사용', 2), ('운영', 2), ('활용', 2), ('개발', 2), ('경험', 2), ('baesg', 1), ('Pyspark', 1), ('MSA', 1), ('지원', 1), ('Docker', 1), ('navercom', 1), ('Swarm', 1), ('Spark', 1), ('Airflow', 1), ('방법', 1), ('hadoop', 1), ('이력서', 1), ('Python', 1)]
[('지원', 2), ('모집', 2), ('채용', 2), ('Sr', 1), ('Data', 1), ('Engineer', 1), ('부문', 1)]
[('개발', 3), ('관련', 2), ('Python', 2), ('기타', 2), ('경험', 2), ('Tool', 2), ('EDW', 1), ('BI', 1), ('구축', 1), ('ETL', 1), ('환경', 1), ('API', 1), ('with', 1), ('프로젝트', 1), ('Stored', 1), ('분석', 1), ('사용', 1), ('SQL', 1), ('GCP', 1), ('Procedure', 1)]
[('개발', 78), ('경험', 63), ('직무', 44), ('직업', 42), ('분야', 37), ('기술', 31), ('데이터', 30), ('마케팅', 29), ('전문', 27), ('IT', 27), ('담당', 26), ('데이터', 26), ('가능', 24), ('홍보', 23), ('우대', 23), ('영업', 22), ('조사', 22), ('의료', 21), ('합니다', 21), ('이상', 20), ('데이터', 20), ('관리', 19), ('스택', 19), ('업무', 19), ('사용', 18), ('사항', 17), ('자격', 17), ('경력', 17), ('학력', 17), ('요건', 17), ('데이터', 15), ('보유', 15), ('지원', 14), ('데

In [ ]:
# import itertools


# m_d = set(d for d in list(itertools.chain.from_iterable(temp_main)) if isinstance(d,str)and len(d)>1)
# r_d = set(d for d in list(itertools.chain.from_iterable(temp_require)) if isinstance(d,str)and len(d)>1)
# c_d = set(d for d in list(itertools.chain.from_iterable(temp_common)) if isinstance(d,str)and len(d)>1)
# p_d = set(d for d in list(itertools.chain.from_iterable(temp_pt)) if isinstance(d,str)and len(d)>1)
# m_d,r_d,c_d,p_d